In [37]:
import pandas as pd
import numpy as np
import re
df=pd.read_csv('../Emission_computing/final_ingredients_emissions.csv',error_bad_lines=False)
from os import name
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer 
import re
nltk.download('wordnet')

/home/mateo/.pyenv/versions/3.8.12/envs/FritzWebInterface/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 280: expected 2 fields, saw 4\n'
[nltk_data] Downloading package wordnet to /home/mateo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [38]:
df

,ingredient,emissions
0,allspice,1133.0
1,almond extract,656.0
2,almonds,2856.0
3,anchovy,4220.0
4,apple juice,2440.0
...,...,...
555,bok choy,622.0
556,figs,192.0
557,mung bean sprouts,428.0
558,soup mix,428.0


In [39]:
output_dict={'ingredient': ['ground beef', 'fresh ginger', 'garlic', 'cilantro', 'limes', 'salt', 'chile paste', 'red onion', 'sea bass', 'chili', 'cilantro', 'chili', 'sweet potato', 'fine sea salt'], 'value': [0.0, 0.06, 0.25, 1.0, 2.0, 0.12, 0.5, 0.25, 0.33, 0.0, 0.25, 0.25, 0.25, 0.0], 'metric': [None, 'piece', 'clove', 'sprig', '<unit>', 'teaspoon', 'teaspoon', '<unit>', 'pound', None, 'sprig', '<unit>', '<unit>', None]}

In [40]:
output_dict

{'ingredient': ['ground beef',
  'fresh ginger',
  'garlic',
  'cilantro',
  'limes',
  'salt',
  'chile paste',
  'red onion',
  'sea bass',
  'chili',
  'cilantro',
  'chili',
  'sweet potato',
  'fine sea salt'],
 'value': [0.0,
  0.06,
  0.25,
  1.0,
  2.0,
  0.12,
  0.5,
  0.25,
  0.33,
  0.0,
  0.25,
  0.25,
  0.25,
  0.0],
 'metric': [None,
  'piece',
  'clove',
  'sprig',
  '<unit>',
  'teaspoon',
  'teaspoon',
  '<unit>',
  'pound',
  None,
  'sprig',
  '<unit>',
  '<unit>',
  None]}

In [41]:
missing_values = ['None']

In [42]:
def fill_empties(output_dict):
    output_df=pd.DataFrame(output_dict)
    output_df.metric=output_df.metric.astype(str)
    for item in missing_values:
        output_df.metric.replace(item, "g", inplace=True)
    return output_df

In [68]:
output_df=fill_empties(output_dict)

In [72]:
for ingredient in output_df['ingredient']:
    # 1. Lemmatize 
    ingredient_words=re.split('\s+', ingredient)
    lemmatizer = WordNetLemmatizer()
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in ingredient_words])
    print([lemmatized_output])

['ground beef']
['fresh ginger']
['garlic']
['cilantro']
['lime']
['salt']
['chile paste']
['red onion']
['sea bass']
['chili']
['cilantro']
['chili']
['sweet potato']
['fine sea salt']


In [ ]:
if df[df['ingredient'].str.match(r'.*'+str(lemmatized_output)+'.*')== True].ingredient.values.size>0:
    final_dict['ingredient'].append(df[df['ingredient'].str.match(r'.*'+str(lemmatized_output)+'.*')== True].ingredient.values[0])
    final_dict['emission'].append(df[df['ingredient'].str.match(r'.*'+str(lemmatized_output)+'.*')== True].emissions.values[0])
        

In [ ]:
import pdb; 

--Call--
> /home/mateo/.pyenv/versions/3.8.12/envs/FritzWebInterface/lib/python3.8/site-packages/IPython/core/displayhook.py(252)__call__()
    250         sys.stdout.flush()
    251 
--> 252     def __call__(self, result=None):
    253         """Printing with history cache management.
    254 

ipdb> next
> /home/mateo/.pyenv/versions/3.8.12/envs/FritzWebInterface/lib/python3.8/site-packages/IPython/core/displayhook.py(258)__call__()
    256         activated by setting the variable sys.displayhook to it.
    257         """
--> 258         self.check_for_underscore()
    259         if result is not None and not self.quiet():
    260             self.start_displayhook()

ipdb> next
> /home/mateo/.pyenv/versions/3.8.12/envs/FritzWebInterface/lib/python3.8/site-packages/IPython/core/displayhook.py(259)__call__()
    257         """
    258         self.check_for_underscore()
--> 259         if result is not None and not self.quiet():
    260             self.start_displayhook()
    26

In [ ]:
def match_ingredients(output_df):
    final_dict={'ingredient':[], 'emission':[]}
    missing_ingredients={'missing_ingredients':[]}
    for ingredient in output_df['ingredient']:
        # 1. Lemmatize 
        ingredient_words=re.split('\s+', ingredient)
        lemmatizer = WordNetLemmatizer()
        lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in ingredient_words])
        
        # 2. Try the whole lemmatized sentence
        if df[df['ingredient'].str.match(r'.*'+str(lemmatized_output)+'.*')== True].ingredient.values.size>0:
                final_dict['ingredient'].append(df[df['ingredient'].str.match(r'.*'+str(lemmatized_output)+'.*')== True].ingredient.values[0])
                final_dict['emission'].append(df[df['ingredient'].str.match(r'.*'+str(lemmatized_output)+'.*')== True].emissions.values[0])
        
        # 3. If the lemmatized output does not work, try the words
        pdb.set_trace()
        else:
            for word in ingredient_words:
                try:
                    final_dict['ingredient'].append(df[df['ingredient'].str.match(r'.*'+str(word)+'.*')== True].ingredient.values[0])
                    final_dict['emission'].append(df[df['ingredient'].str.match(r'.*'+str(word)+'.*')== True].emissions.values[0])
                except IndexError:
                    missing_ingredients['missing_ingredients'].append(ingredient)
            
    final_df=pd.DataFrame(final_dict)
    
    
    # Adding the columns "value" and metric from output_dict
    final_df=final_df.merge(output_df,on="ingredient")
#     final_df['value']=output_df['value']
#     final_df['metric']=output_df['metric']
    return final_df,missing_ingredients

In [44]:
def convert(final_df):
    output_df=final_df.merge(conv_df, on='metric')
    output_df["calculated gCO2e"]=output_df["emission"]*output_df["value"].astype('int64')*output_df["to_Kg_multiplier"]    
    return output_df

In [45]:
output_df=fill_empties(output_dict)

In [ ]:
final_df,missing_ingredients=match_ingredients(output_df)

In [47]:
final_df

,ingredient,emission,value,metric
0,fresh ginger,452.0,0.06,piece
1,garlic,452.0,0.25,clove
2,cilantro,1133.0,1.00,sprig
3,cilantro,1133.0,0.25,sprig
4,cilantro,1133.0,1.00,sprig
5,cilantro,1133.0,0.25,sprig
6,salt,1133.0,0.12,teaspoon
7,salt,1133.0,0.12,teaspoon
8,chile paste,1535.0,0.50,teaspoon
9,red onion,1051.0,0.25,<unit>


In [50]:
ingredient_words=re.split('\s+', "ground beef")
lemmatizer = WordNetLemmatizer()
lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in ingredient_words])

In [51]:
lemmatized_output

'ground beef'

In [59]:
for word in lemmatized_output:


In [60]:
df[df['ingredient'].str.match(r'.*'+str('ground')+'.*')== True].ingredient.values[0]

'groundnut oil'

In [64]:
df[df['ingredient'].str.match(r'.*'+str('beef')+'.*')== True].emissions.values[0]

43330.0

In [ ]:
re.split('\s+', "cherry tomatoes")

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer 

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
print(lemmatizer.lemmatize("sweet potatoes"))

In [ ]:
nltk.download('wordnet')

In [ ]:
if df[df['ingredient'].str.match(r'^'+str("word")+'.*')== True].ingredient.values.size>0:
    print('OK')
else:
    print('Not OK')

# Solving the API issue

In [ ]:
def load_image(img):
    im = Image.open(img)
    image = im.resize((224,224))    

    return image

In [ ]:
img=load_image("../2014_Bruschetta_The_Larder_Chiang_Mai.jpg")

In [ ]:
img=np.array(img)

In [ ]:
img.shape

In [ ]:
import PIL
from PIL import Image
import numpy as np
import json

In [ ]:
X = img.reshape(img.shape[0]*img.shape[1]*img.shape[2])
X=X.tolist()
X_json=json.dumps(X)

In [ ]:
import pickle

In [ ]:
filehandler = open(b"../Image.obj","wb")
# pickle.dump(response_reshape,filehandler)

In [ ]:
filehandler